In [ ]:
import pandas as pd

In [ ]:
import os
import pandas as pd

# Specify the folder containing your Excel files
data_folder = "Data"

# Define the column names
col_names = ['Case ID', 'Suspect Product Active Ingredients', 'Reason for Use', 'Reactions', 'Serious', 'Outcomes', 'Sex', 'Patient Age', 'Patient Weight']

# Initialize an empty DataFrame to store the combined data
combined_data = pd.DataFrame(columns=col_names)

# Iterate over all Excel files in the folder
for file in os.listdir(data_folder):
    if file.endswith(".xlsx"):  # Check if the file is an Excel file
        file_path = os.path.join(data_folder, file)
        # Read the Excel file and add it to the combined DataFrame
        data = pd.read_excel(file_path, usecols=col_names)  # Load only the specified columns
        combined_data = pd.concat([combined_data, data], ignore_index=True)

# Display the combined dataset
print(combined_data)

In [ ]:
col_names = ['Case ID', 'Suspect Product Active Ingredients', 'Reason for Use', 'Reactions', 'Serious', 'Outcomes', 'Sex', 'Patient Age', 'Patient Weight']

# load dataset
data = pd.read_excel("Data\\INSULIN BEEF (G).xlsx")

In [ ]:
data.head()

In [ ]:
selected_columns = ['Case ID', 'Suspect Product Active Ingredients', 'Reactions',  'Serious', 'Sex', 'Patient Age', 'Patient Weight']

#df = data[selected_columns]
df = combined_data[selected_columns]

df.head()
print(df)

In [ ]:
df['Suspect Product Active Ingredients'] = df['Suspect Product Active Ingredients'].str.split(';')
df_split_temp = df.explode('Suspect Product Active Ingredients', ignore_index=True)

df_split_temp['Reactions'] = df_split_temp['Reactions'].str.split(';')
df_split = df_split_temp.explode('Reactions', ignore_index = True)

df_split.head()

In [ ]:
count = df_split['Serious'].value_counts()

print(count)

In [ ]:
print(df_split.dtypes)

In [ ]:
df_multi = pd.get_dummies(df_split, columns = ['Suspect Product Active Ingredients', 'Reactions'], prefix=['Product', 'Reaction'], prefix_sep='_')
reactions = [col for col in df_multi.columns if col.startswith("Reaction_")]
df_reaction = df_multi[reactions]
df_multi = df_multi.groupby('Case ID').max().reset_index()

columns_to_exclude = ['Case ID', 'Suspect Product Active Ingredients', 'Reactions', 'Serious', 'Sex', 'Patient Age', 'Patient Weight']

columns_to_convert = [col for col in df_multi.columns if col.startswith("Product_") or col.startswith("Reaction_")] #not in columns_to_exclude]

df_multi[columns_to_convert] = df_multi[columns_to_convert].astype(int)

df_final = df_multi
print(df_multi.columns)
print(df_final)

df_final.head()

In [ ]:
print(df_reaction.columns)

In [ ]:
react = [col.replace("Reaction_", "") for col in df_reaction.columns]

for r in react:
    print(r)

In [ ]:
from collections import defaultdict

# Group reactions by the first word
grouped_reactions = defaultdict(list)

for r in react:
    # Split reaction into words and extract the first word
    first_word = r.split()[0]
    # Group reactions by the first word
    grouped_reactions[first_word].append(r)
i = 0
# Print the grouped reactions
for word, group in grouped_reactions.items():
    i = i + 1
    print(f"{word}: {group}")

print(i)

In [ ]:
df_encoded = df_multi.copy()
print(df_encoded.columns)
#print(df_split['Patient Age'])
df_encoded['Patient Age'] = df_encoded['Patient Age'].astype(str)
df_encoded['Patient Age'] = df_encoded['Patient Age'].str.replace(r'\D+', '', regex=True)
df_encoded['Patient Age'] = pd.to_numeric(df_encoded['Patient Age'], errors='coerce')  # Converts to numeric, sets invalid values to NaN

df_encoded['Patient Weight'] = df_encoded['Patient Weight'].replace('Not Specified', "0 KG")
df_encoded['Patient Weight'] = df_encoded['Patient Weight'].astype(str)
df_encoded['Patient Weight'] = df_encoded['Patient Weight'].str.replace(r'[^\d.]', '', regex=True)
df_encoded['Patient Weight'] = pd.to_numeric(df_encoded['Patient Weight'], errors='coerce')

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

#df_encoded['Suspect Product Active Ingredients'] = label_encoder.fit_transform(df_encoded['Suspect Product Active Ingredients'])
df_encoded['Sex'] = label_encoder.fit_transform(df_encoded['Sex'])
df_encoded['Serious'] = label_encoder.fit_transform(df_encoded['Serious'])
print(df_encoded.isnull().sum())
df_encoded.dropna(inplace=True)
print(df_encoded.isnull().sum())
df_encoded.head()

In [ ]:
count = df_encoded['Serious'].value_counts()

print(count)

In [ ]:
# Sum columns that start with "Reaction_"
reaction_columns = df_encoded.filter(like="Reaction_", axis=1)
column_sums = reaction_columns.sum(axis=0)

print(len(reaction_columns.columns))
# Set the threshold
threshold = 50 #300

# Filter column names with sums greater than the threshold
columns_above_threshold = column_sums[column_sums > threshold].index.tolist()

print(columns_above_threshold)
print(len(columns_above_threshold))

filtered_df = df_encoded[df_encoded[columns_above_threshold].sum(axis=1) > 0]

# Print the number of rows before and after filtering
print(f"Number of rows before filtering: {df_encoded.shape[0]}")
print(f"Number of rows after removing all-zero rows: {filtered_df.shape[0]}")

In [ ]:
cols_to_exclude = ['Case ID', 'Suspect Product Active Ingredients', 'Reactions']#, 'Serious']

feature_cols = [col for col in df_encoded.columns if col not in cols_to_exclude and not col.startswith("Reaction_")]

print(feature_cols)
print(len(feature_cols))

predict_cols = []#["Serious"]
reaction_cols = columns_above_threshold #[col for col in df_encoded.columns if col.startswith("Reaction_")]
predict_cols.extend(reaction_cols)

print(predict_cols)
print(len(predict_cols))

df_filtered = df_encoded[df_encoded[predict_cols].sum(axis=1) > 0]

X = df_filtered[feature_cols]
Y = df_filtered[predict_cols]

#X = df_encoded[feature_cols]
#Y = df_encoded[predict_cols] #df_encoded.Serious

In [ ]:
X.head()


In [ ]:
Y.head()

In [ ]:
import pandas as pd

# Assuming X is a Pandas DataFrame
column_totals = Y.sum()  # Sums each column
for column, total in column_totals.items():
    print(f"Column: {column}, Total: {total}")

In [ ]:

from sklearn.neighbors import NearestNeighbors
import numpy as np

def ml_smote(X, Y, k=5, target_samples=None):
    """
    MLSMOTE implementation for multi-label datasets.
    
    Parameters:
        X (array): Feature matrix (NumPy array or DataFrame).
        Y (array): Multi-label matrix (NumPy array or DataFrame).
        k (int): Number of nearest neighbors.
        n_samples (int): Number of synthetic samples to generate.
    
    Returns:
        X_augmented, Y_augmented: Augmented feature and label matrices.
    """
    # Calculate imbalance ratio per label (IRPL)
    label_frequencies = Y.sum(axis=0)
    irpl = max(label_frequencies) / label_frequencies
    
    # Calculate mean imbalance ratio (MIR)
    mir = np.mean(irpl)
    
    # Identify tail labels (labels with IRPL > MIR)
    tail_labels = np.where(irpl > mir)[0]
    print(tail_labels)
    
    # Identify minority samples
    #minority_indices = np.where(Y.sum(axis=1) < Y.shape[1] / 2)[0]  # Example threshold
    minority_indices = np.where((Y[:, tail_labels].sum(axis=1)) > 0)[0]
    X_minority = X[minority_indices]
    Y_minority = Y[minority_indices]
    
    # Fit nearest neighbors
    knn = NearestNeighbors(n_neighbors=k).fit(X_minority)
    synthetic_X, synthetic_Y = [], []

    # Calculate target samples for each tail label
    if target_samples is None:
        target_samples = (label_frequencies.max() - label_frequencies).astype(int)

    #for _ in range(n_samples):
    for _ in range(target_samples.sum()):
        # Randomly select a minority sample
        idx = np.random.choice(len(X_minority))
        x = X_minority[idx]
        labels = Y_minority[idx]

        # Find k-nearest neighbors
        neighbors = knn.kneighbors([x], return_distance=False)[0]
        #neighbor_idx = np.random.choice(neighbors[1:])  # Avoid selecting the sample itself
        selected_neighbors = neighbors[1:]  # Exclude itself

        # Interpolate between the sample and a neighbor
        #neighbor_x = X_minority[neighbor_idx]
        #synthetic_x = x + np.random.rand() * (neighbor_x - x)
        #synthetic_y = np.logical_or(labels, Y_minority[neighbor_idx]).astype(int)

        # Generate synthetic features by averaging neighbors
        weights = np.random.dirichlet(np.ones(len(selected_neighbors)))
        neighbor_x = np.dot(weights, X_minority[selected_neighbors])
        synthetic_x = x + np.random.rand() * (neighbor_x - x)

        # Generate synthetic labels probabilistically
        synthetic_y = (np.random.rand(len(labels)) < 0.5).astype(int) * labels
        synthetic_y = np.logical_or(synthetic_y, Y_minority[neighbors[1]]).astype(int)

        synthetic_X.append(synthetic_x)
        synthetic_Y.append(synthetic_y)
    
    # Combine original and synthetic data
    X_augmented = np.vstack([X, np.array(synthetic_X)])
    Y_augmented = np.vstack([Y, np.array(synthetic_Y)])
    return X_augmented, Y_augmented

# Example usage with your data
X_resampled, Y_resampled = ml_smote(X.values, Y.values, k=5) #, n_samples=1000)

print("Original data shape:", X.shape, Y.shape)
print("Resampled data shape:", X_resampled.shape, Y_resampled.shape)

In [ ]:

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from collections import Counter
import numpy as np

# Address class imbalance using SMOTE
#smote = SMOTE()
#X_resampled, y_resampled = smote.fit_resample(X, Y)

#X_resampled = X.copy()
#y_resampled = Y.copy()

print(X_resampled.shape)
print(Y_resampled.shape)

# Standardize the data
scaler = StandardScaler()
X_resampled = scaler.fit_transform(X_resampled)

print(X_resampled.shape)
print(Y_resampled.shape)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_resampled, Y_resampled, test_size=0.4, random_state=26)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=26)

#!pip install imbalanced-learn

#from imblearn.over_sampling import SMOTE

#smote = SMOTE(random_state=42)
#X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
import numpy as np

# Count occurrences of 1s for each label (column-wise sum)
label_counts = np.sum(Y, axis=0)
print("Label Counts for Each Label:", label_counts)

# Compute weights inversely proportional to label frequencies
weights = 1.0 / label_counts  # Higher weights for minority labels
print("Sampling Weights for Each Label:", weights)

In [ ]:
import torch
from torch.utils.data import WeightedRandomSampler

# Convert sample weights to a PyTorch tensor
sample_weights_tensor = torch.tensor(sample_weights, dtype=torch.float)

# Create a WeightedRandomSampler
sampler = WeightedRandomSampler(
    weights=sample_weights_tensor,
    num_samples=len(sample_weights_tensor),  # Keep the number of samples the same as your dataset
    replacement=True  # Allows oversampling of minority samples
)

In [ ]:
column_sums = Y_resampled.sum(axis=0)
print("Sum of each column:", column_sums)

In [ ]:
import matplotlib.pyplot as plt
plt.bar(range(len(column_sums)), column_sums)
plt.xlabel("Labels")
plt.ylabel("Occurrences")
plt.title("Label Distribution After Resampling")
plt.show()

In [ ]:
#for column in Y_resampled.columns:
#    print(f"Label: {column}, Distribution: {Y_resampled[column].value_counts()}")

In [ ]:
import warnings
warnings.filterwarnings("ignore")

!pip install torch -q


In [ ]:

#import torch
#import numpy as np
#from torch.utils.data import Dataset, DataLoader

# Convert data to torch tensors
#class Data(Dataset):
#    def __init__(self, X, y):
        #self.X = torch.from_numpy(X.to_numpy().astype(np.float32))
        #self.y = torch.from_numpy(y.to_numpy().astype(np.float32))
#        self.X = torch.from_numpy(X.astype(np.float32))
#        self.y = torch.from_numpy(y.to_numpy().astype(np.float32))
#        self.len = self.X.shape[0]
       
#    def __getitem__(self, index):
#        return self.X[index], self.y[index]
   
#    def __len__(self):
#        return self.len
   
#batch_size = 64

# Instantiate training and test data
#train_data = Data(X_train, y_train)
#train_data = Data(X_resampled, y_resampled)
#train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

#test_data = Data(X_test, y_test)
#test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

#valid_data = Data(X_val, y_val)
#valid_dataloader = DataLoader(dataset=valid_data, batch_size=batch_size, shuffle=False)

# Check it's working
#for batch, (X, y) in enumerate(train_dataloader):
#    print(f"Batch: {batch+1}")
#    print(f"X shape: {X.shape}")
#    print(f"y shape: {y.shape}")
#    break


In [ ]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

# Convert data to torch tensors
class Data(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))  # to_numpy() #Ensure y is a torch tensor
        self.len = self.X.shape[0]

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return self.len


# Compute sample weights for the training data
def compute_sample_weights(y, label_weights):
    # Convert label weights (Pandas Series) to NumPy array
    label_weights_array = label_weights.values
    # Calculate sample weights as the sum of label weights for each row
    sample_weights = np.dot(y, label_weights_array) # .to_numpy()
    return torch.tensor(sample_weights, dtype=torch.float)


# Set batch size
batch_size = 512 # 256 # 128 # 64

# Instantiate training, validation, and test datasets
train_data = Data(X_train, y_train)
valid_data = Data(X_val, y_val)
test_data = Data(X_test, y_test)

# Compute sample weights for the training data
sample_weights = compute_sample_weights(y_train, weights)

# Create a WeightedRandomSampler for the training data
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),  # Ensure all samples are considered
    replacement=True  # Allows oversampling of minority samples
)

# Create DataLoaders for training, validation, and test data
train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, sampler=sampler)  # Use sampler for training
valid_dataloader = DataLoader(dataset=valid_data, batch_size=batch_size, shuffle=False)   # No sampler for validation
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)     # No sampler for testing

# Check that it's working
for batch, (X, y) in enumerate(train_dataloader):
    print(f"Batch: {batch + 1}")
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")
    break

total_label_sums = torch.zeros(y_train.shape[1])  # Adjust shape based on your dataset
for batch_X, batch_Y in train_dataloader:
    total_label_sums += batch_Y.sum(axis=0)
print("Total label distribution across batches:", total_label_sums)

In [ ]:

import torch
from torch import nn
from torch import optim

input_dim = len(feature_cols) #393
hidden_dim = 4096 #400
output_dim = len(predict_cols)

class MultiOutputNeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MultiOutputNeuralNetwork, self).__init__()
        
        # Define multiple layers
        self.layer_1 = nn.Linear(input_dim, hidden_dim // 2 + output_dim)
        nn.init.kaiming_uniform_(self.layer_1.weight, nonlinearity="relu")
        
        self.layer_2 = nn.Linear(hidden_dim // 2 + output_dim, hidden_dim // 4 + output_dim)  # Hidden layer (reduce size)
        nn.init.kaiming_uniform_(self.layer_2.weight, nonlinearity="relu")
        
        self.layer_3 = nn.Linear(hidden_dim // 4 + output_dim, hidden_dim // 8 + output_dim)  # Another hidden layer
        nn.init.kaiming_uniform_(self.layer_3.weight, nonlinearity="relu")

        self.layer_4 = nn.Linear(hidden_dim // 8 + output_dim, hidden_dim // 16 + output_dim)  # Fourth hidden layer
        nn.init.kaiming_uniform_(self.layer_4.weight, nonlinearity="relu")
        
        self.layer_5 = nn.Linear(hidden_dim // 16 + output_dim, hidden_dim // 32 + output_dim)  # Fifth hidden layer
        nn.init.kaiming_uniform_(self.layer_5.weight, nonlinearity="relu")

        self.layer_6 = nn.Linear(hidden_dim // 32 + output_dim, hidden_dim // 64 + output_dim)  # Fourth hidden layer
        nn.init.kaiming_uniform_(self.layer_4.weight, nonlinearity="relu")
        
        self.layer_7 = nn.Linear(hidden_dim // 64 + output_dim, hidden_dim // 128 + output_dim)  # Fifth hidden layer
        nn.init.kaiming_uniform_(self.layer_5.weight, nonlinearity="relu")
        
        self.output_layer = nn.Linear(hidden_dim // 128 + output_dim, output_dim)  # Final output layer

        self.dropout = nn.Dropout(p=0.3)  # Define dropout
        
    def forward(self, x):
        # Pass through layers with ReLU activations
        x = torch.nn.functional.relu(self.layer_1(x))
        x = self.dropout(x)
        x = torch.nn.functional.relu(self.layer_2(x))
        x = self.dropout(x)
        x = torch.nn.functional.relu(self.layer_3(x))
        x = self.dropout(x)
        x = torch.nn.functional.relu(self.layer_4(x))
        x = self.dropout(x)
        x = torch.nn.functional.relu(self.layer_5(x))
        x = self.dropout(x)
        x = torch.nn.functional.relu(self.layer_6(x))
        x = self.dropout(x)
        x = torch.nn.functional.relu(self.layer_7(x))
        x = self.dropout(x)

        #x = torch.nn.functional.sigmoid(self.output_layer(x))  # Sigmoid for binary output
        #x = torch.sigmoid(self.output_layer(x))
        x = self.output_layer(x)
        return x
       
model = MultiOutputNeuralNetwork(input_dim, hidden_dim, output_dim)
print(model)

In [ ]:
import torch
from torch import nn
from torch.nn import BCEWithLogitsLoss


learning_rate = 0.001

label_counts = np.sum(y_train, axis=0)
total_samples = y_train.shape[0]

class_weights = total_samples / (len(label_counts) * label_counts)
class_weights = torch.tensor(class_weights, dtype=torch.float32)

loss_fn = BCEWithLogitsLoss(pos_weight=class_weights)
#loss_fn = nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)



In [ ]:

num_epochs = 50
train_loss_values = []
valid_loss_values = []

train_accuracy_values = []
valid_accuracy_values = []

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for X, y in train_dataloader:
        # zero the parameter gradients
        optimizer.zero_grad()
        #print(X.shape)
       
        # forward pass
        pred = model(X)
        loss = loss_fn(pred, y) #.unsqueeze(-1))
        train_loss += loss.item()

        # Accuracy calculation
        train_correct += (torch.sigmoid(pred) >= 0.5).float().eq(y).sum().item()
        #train_correct += (pred >= 0.5).float().eq(y).sum().item()
        train_total += y.numel()

        # backward pass and optimization
        loss.backward()
        optimizer.step()

    # average training loss for the epoch
    train_loss /= len(train_dataloader)
    train_loss_values.append(train_loss)

    # Training accuracy
    train_accuracy = train_correct / train_total
    train_accuracy_values.append(train_accuracy)

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for X_val, y_val in valid_dataloader:
            pred = model(X_val)
            loss = loss_fn(pred, y_val) #.unsqueeze(-1))
            val_loss += loss.item()

            # Accuracy calculation
            val_correct += (torch.sigmoid(pred) >= 0.5).float().eq(y_val).sum().item()
            #val_correct += (pred >= 0.5).float().eq(y_val).sum().item()
            val_total += y_val.numel()
    
    # Average validation loss for the epoch
    val_loss /= len(valid_dataloader)
    valid_loss_values.append(val_loss)

    # Validation accuracy
    val_accuracy = val_correct / val_total
    valid_accuracy_values.append(val_accuracy)

    # Print loss for the epoch
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")
    print(f"Train Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")

print("Training Complete")

In [ ]:
# Plotting Loss and Accuracy
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(12, 6))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss_values, label='Train Loss')
plt.plot(epochs, valid_loss_values, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Accuracy plot
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracy_values, label='Train Accuracy')
plt.plot(epochs, valid_accuracy_values, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
label_counts = y_train.sum(axis=0)
print("Label counts in training data:", label_counts)

In [ ]:

import matplotlib.pyplot as plt

plt.plot(range(1, num_epochs + 1), train_loss_values, label='Train Loss')
plt.plot(range(1, num_epochs + 1), valid_loss_values, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_curve, classification_report, hamming_loss
from scipy.special import expit  # Sigmoid function
import numpy as np

# Step 1: Dynamically calculate thresholds for all labels
def calculate_thresholds(y_true, y_probs):
    """
    Calculate optimal thresholds for each label based on precision-recall curves.
    
    Parameters:
        y_true (array): True binary labels (shape: [n_samples, n_labels]).
        y_probs (array): Predicted probabilities (shape: [n_samples, n_labels]).
    
    Returns:
        thresholds_array (array): Optimal thresholds for each label.
    """
    thresholds = []
    for i in range(y_true.shape[1]):  # Iterate over all labels dynamically
        precision, recall, thresholds_curve = precision_recall_curve(y_true[:, i], y_probs[:, i])
        f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)  # Avoid division by zero
        optimal_threshold = thresholds_curve[f1_scores.argmax()]  # Find the best threshold
        thresholds.append(optimal_threshold)
    return np.array(thresholds)

# Step 2: Generate predictions using custom thresholds
def apply_thresholds(y_probs, thresholds_array):
    """
    Apply custom thresholds to convert probabilities into binary predictions.
    
    Parameters:
        y_probs (array): Predicted probabilities (shape: [n_samples, n_labels]).
        thresholds_array (array): Thresholds for each label.
    
    Returns:
        y_pred (array): Binary predictions (shape: [n_samples, n_labels]).
    """
    return (y_probs >= thresholds_array).astype(int)

# Use these functions dynamically
with torch.no_grad():
    y_probs = []  # Collect probabilities
    y_true = []  # Collect true labels

    for X, y in test_dataloader:
        outputs = model(X)  # Get raw logits
        #y_probs.append(outputs.numpy())  # Store probabilities
        y_probs.append(expit(outputs.numpy()))  # Convert logits to probabilities
        y_true.append(y.numpy())  # Store true labels

    # Combine batches into full arrays
    y_probs = np.concatenate(y_probs, axis=0)
    y_true = np.concatenate(y_true, axis=0)

    # Dynamically calculate thresholds
    thresholds_array = calculate_thresholds(y_true, y_probs)
    print(thresholds_array)

    # Apply thresholds to generate binary predictions
    y_pred = apply_thresholds(y_probs, thresholds_array)

    # Evaluate the model
    print(classification_report(y_true, y_pred, target_names=[f"Label {i}" for i in range(y_true.shape[1])]))
    print(f"Hamming Loss: {hamming_loss(y_true, y_pred):.4f}")

    # [0.47408065 0.52283853 0.21895869 0.27320343 0.6131553  0.46514297
 #0.47177306 0.4537501  0.3889547  0.44084015 0.37250233 0.672361
 #0.32849404 0.55169785 0.35388562 0.60575116 0.28624716 0.3597172
 #0.34597415 0.3506796  0.53688514 0.31747028 0.77734935 0.31698427
 #0.24684684 0.3295522  0.22854547 0.3962895  0.25093877 0.3283029
 #0.4727016  0.29456106 0.26963988 0.13329153 0.39464808 0.19385336
 #0.3715566  0.11622908 0.1728393  0.50060636 0.13731995 0.24483377
 #0.3754116  0.31347966 0.34662092 0.33132926 0.25891978 0.21420503
 #0.2166132  0.35917073 0.46442243 0.5035977  0.11324815 0.33370334
 #0.30403858 0.7453642  0.30300874 0.5743012  0.25913933 0.76659316
 #0.3750667  0.17950211 0.30545366 0.18148132 0.50782764 0.24113299
# 0.2444831  0.20651104 0.33514607 0.2554716  0.2989948  0.43072158
 #0.47554058 0.35983965 0.42703182 0.1700732  0.1999757  0.63880205
 #0.31359017 0.20798978 0.5765725  0.2324845  0.57795274 0.5981137
 #0.2000306  0.34174556 0.21019131 0.2839584  0.26304948 0.462434
 #0.2133022  0.12184759 0.34505197 0.48796624 0.90137494 0.5976198
 #0.26096636 0.7713822  0.6936902  0.7387786  0.19493434 0.5385473
 #0.08584324 0.23921402 0.47752047 0.2028372  0.17308132 0.21509966
 #0.24524291 0.84455794 0.4313751  0.5592498  0.66809034 0.40178862
 #0.2039171  0.21381447 0.8629443  0.487138   0.6246119  0.7029666
 #0.411958   0.27293777 0.23883899 0.25088048 0.56650364 0.50837135
 #0.11624123 0.424044   0.24882652 0.5561839  0.4825503  0.21246974
 #0.33677018 0.3970621  0.31884047 0.31590977 0.54259104 0.41429722
 #0.35419166]

In [ ]:
import itertools  # Import this at the top of your script

# Initialize required variables
y_pred = []
y_test = []
correct = 0
total = 0

"""
We're not training so we don't need to calculate the gradients for our outputs
"""
with torch.no_grad():
    for X, y in test_dataloader:
        outputs = model(X)  # Get model outputs
        predicted = np.where(outputs.numpy() < 0.5, 0, 1)  # Convert to NumPy and apply threshold
        #predicted = list(itertools.chain(*predicted))  # Flatten predictions
        y_pred.append(predicted)  # Append predictions
        y_test.append(y.numpy())  # Append true labels as NumPy
        total += y.size(0)  # Increment total count
        correct += (predicted == y.numpy()).sum().item()  # Count correct predictions

#print(f'Accuracy of the network on the test instances: {100 * correct // total}%')
y_pred = np.concatenate(y_pred, axis=0)  # Combine batches into a single array
y_test = np.concatenate(y_test, axis=0)  # Combine batches into a single array
total = y_test.size  # Total number of labels
correct = (y_pred == y_test).sum()  # Total number of correct predictions
accuracy = 100 * correct / total
print(f'Accuracy of the network on the test instances: {accuracy:.2f}%')

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import seaborn as sns

#y_pred = list(itertools.chain(*y_pred))
#y_test = list(itertools.chain(*y_test))

from sklearn.metrics import classification_report
from sklearn.metrics import hamming_loss, accuracy_score

# Flatten labels if necessary for multilabel
y_pred_flat = np.array(y_pred).reshape(-1)
y_test_flat = np.array(y_test).reshape(-1)

print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report

# Combine batches of predictions and true labels into a single array
y_pred = []
y_true = []

# Generate predictions for all validation batches
model.eval()
with torch.no_grad():
    for X_val, y_val in valid_dataloader:
        preds = torch.sigmoid(model(X_val))  # Apply sigmoid to get probabilities
        preds = (preds > 0.5).int()  # Convert probabilities to binary predictions
        y_pred.append(preds.numpy())
        y_true.append(y_val.numpy())

# Flatten predictions and true labels
y_pred = np.concatenate(y_pred, axis=0)
y_true = np.concatenate(y_true, axis=0)

# Classification report
print(classification_report(y_true, y_pred, target_names=predict_cols, zero_division=0))

In [ ]:
from sklearn.metrics import hamming_loss, f1_score

# Example for F1-score
#y_pred_binary = (y_pred > 0.5).astype(int)
y_pred_binary = (y_pred > thresholds_array).astype(int)
print("Hamming Loss:", hamming_loss(y_true, y_pred_binary))
print("Micro F1-score:", f1_score(y_true, y_pred_binary, average='micro'))
print("Macro F1-score:", f1_score(y_true, y_pred_binary, average='macro'))

In [ ]:
#cf_matrix = confusion_matrix(y_test, y_pred)
#cf_matrix

In [ ]:
print(type(thresholds_array))

In [ ]:

import numpy as np
import random
import torch

# Exclude specific columns
columns_to_exclude = ['Case ID', 'Suspect Product Active Ingredients', 'Sex', 'Patient Age', 'Patient Weight', 'Serious', 'Reactions']

# Create drug-related columns
drug_cols = [col for col in df_encoded.columns if col.startswith("Product_")] #not in columns_to_exclude]

# Generate random drug data
random_drugs = random.sample(drug_cols, 5)#["Insulin Pork\Insulin Purified Pork", "Insulin Beef"] #random.sample(drug_cols, 3)
print("Patient Drugs: ", random_drugs)
drug_array = np.zeros(len(drug_cols))
column_indices = [df_encoded.columns.get_loc(col) for col in random_drugs]
for idx in column_indices:
    drug_array[idx - 5] = 1  # Adjust index as needed

# Create a specific patient profile
specific_profile = np.array([[1, 1, 83, 74]])  # Adjust profile values as needed
specific_profile = np.concatenate([specific_profile, drug_array.reshape(1, -1)], axis=1)

# Ensure the feature order matches the training data
#columns_to_include = [col for col in X.columns if col not in columns_to_exclude]  # 'X' should be your training feature matrix
#specific_profile = specific_profile[:, :len(columns_to_include)]  # Align shape with training features

# Standardize the profile using the same scaler used during training
specific_profile_scaled = specific_profile #scaler.transform(specific_profile)  # 'scaler' is the StandardScaler from training

# Convert the profile to a PyTorch tensor
specific_profile_tensor = torch.tensor(specific_profile_scaled, dtype=torch.float32)

# Predict the outcome using your PyTorch model
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    prediction = model(specific_profile_tensor)  # Raw logits
    #predicted_outcomes = (torch.sigmoid(prediction) >= 0.5).float()  # Apply sigmoid and threshold
    predicted_probabilities = torch.sigmoid(prediction).numpy()  # Convert logits to probabilities
    #predicted_probabilities = prediction.numpy()  # Convert logits to probabilities
    predicted_outcomes = (predicted_probabilities >= thresholds_array).astype(float)  # Apply sigmoid and threshold

# Output the results for generic labels
#for i, (outcome, probability) in enumerate(zip(predicted_outcomes.flatten(), predicted_probabilities.flatten()), 1):
#    print(f"Label {i}:")
#    print(f"  Occurrence: {'Yes' if outcome == 1 else 'No'}")
#    print(f"  Probability: {probability:.4f}")

# Initialize an empty list to store labels with "Yes" outcomes and their probabilities
#yes_labels_with_probabilities = []

# Iterate over the predicted outcomes and corresponding probabilities
#for i, (outcome, probability) in enumerate(zip(predicted_outcomes.flatten(), prediction.numpy().flatten()), 1):
#    if outcome == 1:  # Check if the outcome is "Yes"
        # Append label and probability to the list
#        yes_labels_with_probabilities.append((f"Label {i}", probability))

# Print the list of labels with "Yes" outcomes and their probabilities
#print("Labels with 'Yes' outcomes and their probabilities:")
#for label, probability in yes_labels_with_probabilities:
#    print(f"{label}: Probability = {probability:.4f}")

# Initialize an empty list to store column names with "Yes" outcomes and their probabilities
yes_labels_with_probabilities = []

# Iterate over the predicted outcomes and corresponding probabilities
for outcome, probability, column_name in zip(
    predicted_outcomes.flatten(), 
    #prediction.numpy().flatten(), 
    torch.sigmoid(prediction).numpy().flatten(),
    Y.columns  
):
    if outcome == 1:  # Check if the outcome is "Yes"
        # Append column name and probability to the list
        yes_labels_with_probabilities.append((column_name, probability))

# Print the list of column names with "Yes" outcomes and their probabilities
print("Columns with 'Yes' outcomes and their probabilities:")
for column_name, probability in yes_labels_with_probabilities:
    print(f"{column_name}: Probability = {probability:.4f}")

In [ ]:
# Check average probabilities for each label
#label_probabilities = prediction.numpy().mean(axis=0)  # Average probabilities across samples
label_probabilities = torch.sigmoid(prediction).mean(axis=0)
label_names = Y.columns  # Assuming these are your column names

for label, prob in zip(label_names, label_probabilities):
    print(f"{label}: Average Probability = {prob:.4f}")